In [34]:
import torch
import torch.nn as nn

In [35]:
inputs = torch.Tensor(1, 1, 28, 28) # 배치사이즈, 채널, 높이, 너비
print('텐서의 크기 : {}'.format(inputs.shape))

텐서의 크기 : torch.Size([1, 1, 28, 28])


In [36]:
# 1채널을 입력받아 32채널을 출력하는데 커널사이즈는3이며 패딩은 1이다.
conv1 = nn.Conv2d(1, 32, 3, padding=1)
print(conv1)

Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))


In [37]:
# 32채널을 입력받아 64채널을 출력하는데 커널사이즈는 3이며 패딩은 1이다.
conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
print(conv2)

Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))


In [38]:
pool = nn.MaxPool2d(2) # 커널사이즈, 스트라이드 입력 정수값으로
print(pool)

MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)


In [39]:
out = conv1(inputs)
print(out.shape)

torch.Size([1, 32, 28, 28])


In [40]:
out = pool(out)
print(out.shape)

torch.Size([1, 32, 14, 14])


In [41]:
out = conv2(out)
print(out.shape)

torch.Size([1, 64, 14, 14])


In [42]:
out = pool(out)
print(out.shape)

torch.Size([1, 64, 7, 7])


In [43]:
out.size(0)

1

In [44]:
out.size(1)

64

In [45]:
out.size(2)

7

In [46]:
out = out.view(out.size(0), -1)
print(out.shape)

torch.Size([1, 3136])


In [47]:
fc = nn.Linear(3136, 10)
out = fc(out)

print(out.shape)

torch.Size([1, 10])


# CNN으로 MNIST 분류

In [48]:
import torch
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import torch.nn.init

In [49]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(777)
if device =='cuda':
    torch.cuda.manual_seed_all(777)

In [50]:
learning_rate = 0.001
training_epochs = 15
batch_size = 100

In [51]:
mnist_train = dsets.MNIST(root='MNIST_data/',
                         train=True,
                         transform = transforms.ToTensor(),
                         download=True)

mnist_test = dsets.MNIST(root = 'MNIST_data/',
                        train = False,
                        transform = transforms.ToTensor(),
                        download=True)

In [53]:
data_loader = torch.utils.data.DataLoader(dataset = mnist_train,
                                    batch_size = batch_size,
                                    shuffle = True,
                                    drop_last = True)

In [64]:
class CNN(torch.nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        
        # 첫번째층
        # ImgIn shape=(?, 28, 28, 1)
        #    Conv     -> (?, 28, 28, 32)
        #    Pool     -> (?, 14, 14, 32)
        self.layer1 = torch.nn.Sequential(
            torch.nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2))

        # 두번째층
        # ImgIn shape=(?, 14, 14, 32)
        #    Conv      ->(?, 14, 14, 64)
        #    Pool      ->(?, 7, 7, 64)
        self.layer2 = torch.nn.Sequential(
            torch.nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2))

        # 전결합층 7x7x64 inputs -> 10 outputs
        self.fc = torch.nn.Linear(7 * 7 * 64, 10, bias = True)
        
        torch.nn.init.xavier_uniform_(self.fc.weight)
        
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.view(out.size(0), -1)
        out = self.fc(out)
        return out

In [65]:
model = CNN().to(device)

In [66]:
criterion = torch.nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [67]:
total_batch = len(data_loader)
print('총 배치의 수 : {}'.format(total_batch))

총 배치의 수 : 600


In [68]:
for epoch in range(traing_epochs):
    avg_cost = 0
    
    for X, Y in data_loader:
        X = X.to(device)
        Y = Y.to(device)
        
        optimizer.zero_grad()
        
        hypothesis = model(X)
        cost = criterion(hypothesis, Y)
        cost.backward()
        
        optimizer.step()
        
        avg_cost += cost / total_batch
        
    print('[Epoch : {:>4}] cost = {:>.9}'.format(epoch + 1, avg_cost))

[Epoch :    1] cost = 0.209968925
[Epoch :    2] cost = 0.0609945841
[Epoch :    3] cost = 0.0455990918
[Epoch :    4] cost = 0.0369366184
[Epoch :    5] cost = 0.0311175194
[Epoch :    6] cost = 0.0261252429
[Epoch :    7] cost = 0.0222429764
[Epoch :    8] cost = 0.0180698857
[Epoch :    9] cost = 0.0151674496
[Epoch :   10] cost = 0.0134097645
[Epoch :   11] cost = 0.0118873855
[Epoch :   12] cost = 0.0103153056
[Epoch :   13] cost = 0.00814865809
[Epoch :   14] cost = 0.00670190202
[Epoch :   15] cost = 0.0065228031


In [69]:
with torch.no_grad():
    X_test = mnist_test.test_data.view(len(mnist_test), 1, 28, 28).float().to(device)
    Y_test = mnist_test.test_labels.to(device)
    
    prediction = model(X_test)
    correct_prediction = torch.argmax(prediction, 1) == Y_test
    accuracy = correct_prediction.float().mean()
    print('Accuracy : ', accuracy.item())

Accuracy :  0.986299991607666
